In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

In [4]:
df = pd.read_csv("train.csv")
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
586,587,0,2,"Jarvis, Mr. John Denzil",male,47.0,0,0,237565,15.0000,NaN,S
680,681,0,3,"Peters, Miss. Katie",female,NaN,0,0,330935,8.1375,NaN,Q
467,468,0,1,"Smart, Mr. John Montgomery",male,56.0,0,0,113792,26.5500,NaN,S
361,362,0,2,"del Carlo, Mr. Sebastiano",male,29.0,1,0,SC/PARIS 2167,27.7208,NaN,C
49,50,0,3,"Arnold-Franchi, Mrs. Josef (Josefine Franchi)",female,18.0,1,0,349237,17.8000,NaN,S


In [5]:
df.drop(columns=["Name","PassengerId","Cabin","Ticket"],inplace= True)
df.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
468,0,3,male,NaN,0,0,7.7250,Q
433,0,3,male,17.0,0,0,7.1250,S
256,1,1,female,NaN,0,0,79.2000,C
676,0,3,male,24.5,0,0,8.0500,S
780,1,3,female,13.0,0,0,7.2292,C


In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(df.drop("Survived", axis =1),df["Survived"],test_size = 0.2,random_state =2)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [7]:
X_train.sample(1)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
421,3,male,21.0,0,0,7.7333,Q


In [8]:
#Imputation of age and Embarked

si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

X_train_age = si_age.fit_transform(X_train[["Age"]])
X_train_embarked = si_embarked.fit_transform(X_train[["Embarked"]])
X_test_age = si_age.fit_transform(X_test[["Age"]])
X_test_embarked = si_embarked.fit_transform(X_test[["Embarked"]])


In [9]:
# onehot encoding of sex and Embarked
ohe_sex = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse=False, handle_unknown='ignore')

X_train_sex = ohe_sex.fit_transform(X_train[["Sex"]])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[["Sex"]])
X_test_embarked = ohe_embarked.transform(X_test_embarked)


In [10]:
X_train_sex.shape ,X_train_embarked.shape,X_test_sex.shape,X_test_embarked.shape

((712, 2), (712, 3), (179, 2), (179, 3))

In [11]:
X_train_rem = X_train.drop(columns=["Age","Sex","Embarked"])
X_test_rem = X_test.drop(columns=["Age","Sex","Embarked"])


In [19]:
X_train_rem

,Pclass,SibSp,Parch,Fare
30,1,0,0,27.7208
10,3,1,1,16.7000
873,3,0,0,9.0000
182,3,4,2,31.3875
876,3,0,0,9.8458
...,...,...,...,...
534,3,0,0,8.6625
584,3,0,0,8.7125
493,1,0,0,49.5042
527,1,0,0,221.7792


In [12]:
X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embarked), axis = 1)
X_test_transformed = np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embarked), axis =1)

In [20]:
X_train_transformed.shape

(712, 10)

In [14]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,Y_train)

DecisionTreeClassifier()

In [15]:
Y_Pred = clf.predict(X_test_transformed)

In [16]:
from sklearn.metrics import accuracy_score
score =  accuracy_score(Y_test,Y_Pred)
score

0.776536312849162

In [17]:
import pickle

In [18]:
pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('models/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('models/clf.pkl','wb'))